In [1]:
import pandas as pd
import dotenv

dotenv.load_dotenv()

True

In [2]:
df = pd.read_csv("./product_sentiment_polarity/data/samsung-labeled-transformed.csv")
df.head()

,question,sentiment,answer
0,What are the top 3 benefits of Galaxy AI?,very_positive,The top 3 benefits of the Galaxy AI are: unpar...
1,What are the main differences between the Sams...,very_positive,The main differences between the Samsung Galax...
2,Does the Samsung Galaxy S23 Ultra support 8K v...,very_positive,Absolutely! The Samsung Galaxy S23 Ultra does ...
3,How does the battery life of the Galaxy S23+ c...,very_positive,The Galaxy S23+ offers a remarkable battery li...
4,Can I use a stylus with the Samsung Galaxy S23...,very_positive,Absolutely! The Samsung Galaxy S23 Ultra is de...


In [ ]:
from sklearn.model_selection import train_test_split
import dspy

# Load math questions from the GSM8K dataset
dataset = []
for _, row in df.iterrows():
    dataset.append(
        dspy.Example(output=row.answer, sentiment=row.sentiment).with_inputs("output")
    )
trainset, devset = train_test_split(dataset, train_size=50, test_size=0.1)
print("trainset len", len(trainset))
trainset[0:10]

In [35]:
import json
from typing import Any

model = "gpt-3.5-turbo"

# Set up the LM
llm = dspy.OpenAI(
    model=model,
    max_tokens=2048,
    tools=[
        {
            "type": "function",
            "function": {
                "name": "sentiment",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "positive_points": {
                            "type": "string",
                            "description": "positive points mentioned on the output, empty string if none",
                        },
                        "negative_points": {
                            "type": "string",
                            "description": "negative points mentioned on the output, empty string if none",
                        },
                        "sentiment": {
                            "type": "string",
                            "enum": [
                                "very_positive",
                                "subtly_positive",
                                "subtly_negative",
                                "very_negative",
                            ],
                            "description": "the sentiment of the output following one of the 4 options",
                        },
                    },
                    "required": ["positive_points", "negative_points", "sentiment"],
                },
                "description": "use this function if you need to give your veredict on the sentiment",
            },
        },
    ],
    temperature=0,
    # tool_choice="auto",
    tool_choice={"type": "function", "function": {"name": "sentiment"}},
)


def _get_choice_text(self, choice: dict[str, Any]) -> str:
    prompt: str = self.history[-1]["prompt"]
    # print("\n\nprompt", prompt, "\n\n")
    if self.model_type == "chat":
        message = choice["message"]
        if content := message["content"]:
            return content
        elif tool_calls := message.get("tool_calls", None):
            arguments = json.loads(tool_calls[0]["function"]["arguments"])
            if prompt.endswith("Reasoning:"):
                return arguments["reasoning"] + "\nSentiment: " + arguments["sentiment"]
            if prompt.strip().endswith("Positive Points:"):
                return (
                    ('None' if arguments["positive_points"] == "" else arguments["positive_points"])
                    + "\n\nNegative Points: "
                    + ('None' if arguments["negative_points"] == "" else arguments["negative_points"])
                    + "\n\nSentiment: "
                    + arguments["sentiment"]
                )
            else:
                return arguments["sentiment"]
    return choice["text"]


llm._get_choice_text = _get_choice_text.__get__(llm)

print(llm("hello there, Positive Points:")[0])

dspy.settings.configure(lm=llm)

Hello there

Negative Points: None

Sentiment: subtly_positive


In [36]:
from typing import Literal

from pydantic import BaseModel, Field


class Sentiment(BaseModel):
    sentiment: Literal["very_positive", "subtly_positive", "subtly_negative", "very_negative"] = Field(description="The sentiment of the output following one of the 4 options")

class ProductSentimentPolaritySignature(dspy.Signature):
    """Classify the sentiment of the output."""

    output = dspy.InputField(desc="Output of LLM talking about product specifications.")
    positive_points : str = dspy.OutputField(desc="Positive points mentioned on the output.")
    negative_points : str = dspy.OutputField(desc="Negative points mentioned on the output.")
    sentiment : Sentiment = dspy.OutputField()

class ProductSentimentPolarity(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.Predict(ProductSentimentPolaritySignature)

    def forward(self, output):
        return self.prog(output=output)

dev_example = devset[0]
print(dev_example)

pred = ProductSentimentPolarity()(output=dev_example.output)
pred

Example({'output': "Yes, the Galaxy A34 is compatible with 5G networks. However, don't get too excited. Samsung's software often throttles performance, leading to inconsistent network speeds and stability issues. Plus, they tend to neglect timely software updates for mid-range models. So, the 5G feature might not deliver the seamless experience you expect.", 'sentiment': 'very_negative'}) (input_keys={'output'})


Prediction(
    positive_points='None',
    negative_points="Samsung's software throttles performance, inconsistent network speeds, stability issues, neglect timely software updates for mid-range models",
    sentiment='very_negative'
)

In [37]:
from dspy.evaluate import Evaluate

def sentiment_matches(example, pred, trace=None):
    return example.sentiment == pred.sentiment

scores = []

for i in range(0, 10):
    evaluation = Evaluate(
        devset=trainset, metric=sentiment_matches, num_threads=16, display_progress=True
    )
    score = evaluation(ProductSentimentPolarity())  # type: ignore
    scores.append(score)

# statistics summay
import numpy as np

print("average score", sum(scores) / len(scores))
print("median score", np.median(scores))
print("min score", min(scores))
print("max score", max(scores))
print("variance", np.var(scores))

Average Metric: 33 / 50  (66.0): 100%|██████████| 50/50 [00:04<00:00, 10.17it/s]

average score 63.2
median score 62.0
min score 60.0
max score 68.0
variance 4.96


In [67]:
# %cd /Users/rchaves/Projects/langwatch-saas/langwatch/python-sdk/
# %pip install .

import langwatch

langwatch.endpoint = "http://localhost:3000"
langwatch.login()

LangWatch API key is already set, if you want to login again, please call as langwatch.login(relogin=True)


In [72]:
from dspy.teleprompt import BootstrapFewShot, BootstrapFewShotWithRandomSearch


optimizer = BootstrapFewShotWithRandomSearch(
    metric=sentiment_matches,
    max_bootstrapped_demos=16,
    max_labeled_demos=4,
    max_rounds=1,
    num_candidate_programs=16,
)

langwatch.dspy.init(experiment="product_sentiment_polarity_openai", optimizer=optimizer)

optimized_evaluator = optimizer.compile(ProductSentimentPolarity(), trainset=trainset)

Experiment initialized, run_id: dashing-eccentric-chowchow
Open http://localhost:3000/inbox-narrator/experiments/product_sentiment_polarity_v1?runIds=dashing-eccentric-chowchow to track your DSPy training session live



Average Metric: 39 / 50  (78.0): 100%|██████████| 50/50 [00:12<00:00,  3.88it/s] 


In [80]:
from dspy.evaluate import Evaluate

# Set up the evaluator, which can be used multiple times.
evaluate_train = Evaluate(devset=trainset, metric=sentiment_matches, num_threads=4, display_progress=True, display_table=0)
evaluate_dev = Evaluate(devset=devset, metric=sentiment_matches, num_threads=4, display_progress=True, display_table=0)

# Evaluate our `optimized_cot` program.
train_score = evaluate_train(optimized_evaluator)
dev_score = evaluate_dev(optimized_evaluator)
train_score, dev_score

Average Metric: 36 / 41  (87.8): 100%|██████████| 41/41 [00:00<00:00, 3561.78it/s] 


(86.0, 87.8)

In [84]:
import os
if not os.path.exists('./results'):
    os.makedirs('./results')
optimized_evaluator.save(f"./results/{langwatch.dspy.experiment_slug}_{model}_{langwatch.dspy.run_id}_train_{train_score}_dev_{dev_score}.json")

In [ ]:
from tqdm import tqdm
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# optimized_evaluator = ProductSentimentPolarity()
# optimized_evaluator.load("./product_sentiment_polarity_anthropic_no_reasoning_v1_80.json")
# optimized_evaluator(output=dev_example.output)

evaluate = Evaluate(devset=devset, metric=sentiment_matches, num_threads=4, display_progress=True, display_table=0, return_outputs=True)
score, results = evaluate(optimized_evaluator) # type: ignore

sentiment_map = {
  "very_positive": 3,
  "subtly_positive": 2,
  "subtly_negative": 1,
  "very_negative": 0
}

y = [sentiment_map[example[0].sentiment] for example in results]
y_pred = [sentiment_map[example[1].sentiment] for example in results]


conf_matrix = confusion_matrix(y, y_pred)

# Plotting confusion matrices
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', ax=ax)
ax.set_title('Confusion Matrix: Actual vs Predicted')
ax.set_xlabel('Predicted')
ax.set_ylabel('Actual')

plt.tight_layout()
